In [ ]:
# default_exp modules.files

# export
import collections

import typing
import traitlets
from traitlets import HasTraits, Dict, Any, Unicode, Integer, Bytes
from dharpa_toolbox.modules.core import DharpaModule, ModuleInputValues, ModuleOutputValues

class DharpaFile(HasTraits):

    @classmethod
    def create(self, data):

        print("--")
        print(data)

        if isinstance(data, str):
            raise NotImplementedError()

        if not isinstance(data, collections.abc.Mapping) and isinstance(data, collections.abc.Iterable) and len(data) == 1:
            data = list(data)[0]

        if isinstance(data, collections.abc.Mapping):

            if len(data) == 1:
                value = next(iter(data.values()))

                if isinstance(value, collections.abc.Mapping) and isinstance(value.get("metadata", None), collections.abc.Mapping) and isinstance(value.get("content", None), bytes):
                    # means uploaded file from FileUpload widget
                    # TODO: maybe check file name just to be sure
                    data = value

            if "metadata" in data.keys() and "content" in data.keys():
                name = data["metadata"]["name"]
                size = data["metadata"]["size"]
                f_type = data["metadata"].get("type", None)
                last_modified = data["metadata"]["lastModified"]
            else:
                raise ValueError(f"Can't parse dict to file object: {data}")
        else:
            raise TypeError(f"Can't create file object: invalid input type '{type(data)}'")

        f = DharpaFile(name=name, size=size, type=f_type, last_modified=last_modified)
        return f

    name = Unicode()
    type = Unicode(allow_none=True)
    size = Integer()
    last_modified = Integer(allow_none=True)
    content = Bytes()


class DharpaFiles(object):

    @classmethod
    def create(self, data) -> typing.List["DharpaFile"]:

        result = []

        if not data:
            raise ValueError("No data provided for file object.")

        if isinstance(data, str):
            raise NotImplementedError()
        if isinstance(data, collections.abc.Mapping):
            first_key = next(iter(data.keys()))
            if isinstance(first_key, str) and isinstance(data[first_key], collections.abc.Mapping) and isinstance(data[first_key].get("metadata", None), collections.abc.Mapping) and isinstance(data[first_key].get("content", None), bytes):

                for v in data.values():
                    f = DharpaFile.create(v)
                    result.append(f)
            else:
                f = DharpaFile.create(data)
                result.append(f)

        elif isinstance(data, collections.abc.Iterable):
            for d in data:
                f = DharpaFile.create(d)
                result.append(f)

        return DharpaFiles(*result)

    def __init__(self, *files: DharpaFile):

        self._files: typing.Iterable[DharpaFile] = files


class FilesCollectionModule(DharpaModule):

    _module_name = "files_collection"


    def _create_inputs(self, **config) -> ModuleInputValues:

        class FilesCollectionValueInput(ModuleInputValues):
            files = traitlets.Any()

        return FilesCollectionValueInput()

    def _create_outputs(self, **config) -> ModuleOutputValues:

        class FilesCollectionValueOutput(ModuleOutputValues):
            files = traitlets.Instance(klass=DharpaFiles, allow_none=True)

        return FilesCollectionValueOutput()

    def _process(self, files):

        print("----")

        print(files)

        d_files = DharpaFiles.create(files)

        print(d_files)

        return {"files": d_files}

    # def _create_widget(self, widget_name):
    #
    #     if widget_name == "upload":
    #         file_upload = FileUpload(accept=".csv", multiple=False)
    #
    #         def set_module_value(change):
    #
    #             m = []
    #             for v in change.new.values():
    #                 m.append(v)
    #
    #             self.inputs.files.extend(m)
    #             # file_upload.value.clear()
    #             file_upload._counter = 0
    #             self.process()
    #
    #         file_upload.observe(set_module_value, names="value")
    #         return file_upload
    #
    #     if widget_name == "file_list":
    #         file_list = Output()
    #
    #         with file_list:
    #             print("file list placeholder")
    #
    #         def update_file_list(change):
    #
    #             file_list.clear_output()
    #             with file_list:
    #                 display(self.outputs.files)
    #
    #         self.outputs.observe(update_file_list, names="files")
    #
    #         return file_list

In [ ]:
from dharpa_toolbox.core import export_notebooks
export_notebooks()

Converted 00_core.ipynb.
Converted 01_modules_core.ipynb.
Converted 02_data_sources.ipynb.
Converted 03_data_targets.ipynb.
Converted 05_workflows.ipynb.
Converted 10_module_subclasses.ipynb.
Converted 10_module_subclasses_old.ipynb.
Converted 99_utils.ipynb.
